___
# Ciência dos Dados - PROJETO 2

___
## Guilherme Leite

## Leonardo Neves

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [39]:
import pandas as pd
import os
import numpy as np

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Leonardo\OneDrive\insper\dps_2018\ciencia_dos_dados\projeto_2\DPCD-master\P2


In [73]:
df = pd.read_excel('spamham.xlsx') #le o arquivo

df = df.replace(',','',regex=True)
df = df.replace('@','',regex=True)
df = df.replace(';','',regex=True)
df = df.replace('  ','',regex=True)
df = df.replace(':','',regex=True)
df = df.replace('"','',regex=True)
df = df.replace('\\*','',regex=True)
df = df.replace('\*','',regex=True)
df = df.replace('_','',regex=True)
df = df.replace('\$','',regex=True)
df = df.replace('rt','',regex=True)
df = df.replace('//','',regex=True)
df = df.replace('/','',regex=True)
df = df.replace('#','',regex=True)
df = df.replace('\(','',regex=True)
df = df.replace('\)','',regex=True)
df = df.replace('\'','',regex=True)
df = df.replace('\?','',regex=True)
df = df.replace('\-','',regex=True)
df = df.replace('!','',regex=True)

df = df.sample(frac=1) #Deixa o df em uma ordem aleatoria

L = len(df)
Learn = int(L*0.75) #tamanho do df de treinamento
Test = int(L*0.25) # tamenho do df de teste

dfLearn = df.iloc[0:Learn] #df de treinamento
dfLearn = dfLearn.reset_index(drop=True)

dfTest = df.iloc[Learn:L] #df de teste
dfTest = dfTest.reset_index(drop=True)


In [74]:
Ham = []
Spam =[]
NSpam = 0

for index, row in dfLearn.iterrows(): #Percorre o df linha por linha
    if row['Class'] == 'spam':
        linha = row['Email']
        Linha = linha.split() #Separa as palavras do conteúdo do email
        NSpam +=1 #Adiciona +1 ao numero de emails de Spam
        for palavra in Linha:
            Hm = []
            for letra in palavra:  #Separa as letras das palavras
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra) 
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                Spam.append(hm)
    else:
        linha = str(row['Email'])
        Linha = linha.split()  #Separa as palavras do conteúdo do email
        for palavra in Linha:
            Hm = []
            for letra in palavra:
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra)
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                Ham.append(hm)

In [75]:
print(Spam[0:10])

['Hi', 'its', 'LUCY', 'Hubby', 'at', 'meetins', 'all', 'day', 'Fri', 'will']


___
## 3. Classificador Naive-Bayes

In [76]:
P_Spam = NSpam/len(dfLearn) #Probabilidade de uma mensagem ser SPAM 
P_Ham = 1-P_Spam #Probabilidade de uma mensagem ser HAM
print('Probabilidade de uma mensagem ser SPAM: ',P_Spam, 
      '\nProbabilidade de uma mensagem ser HAM: ',P_Ham)

Probabilidade de uma mensagem ser SPAM:  0.1255980861244019 
Probabilidade de uma mensagem ser HAM:  0.8744019138755981


In [115]:
d_ham =dfTest[dfTest.Class == 'ham']
d_spam =dfTest[dfTest.Class == 'spam']
d_ham.Email

2       Do you want a New Nokia 3510i colour phone Del...
3       87077 Kick off a new season with 2wks FREE goa...
4       44 7732584351 Do you want a New Nokia 3510i co...
22      09066362231 URGENT Your mobile No 07xxxxxxxxx ...
44      Eerie Nokia tones 4u rply TONE TITLE to 8007 e...
46      cmon babe make me horny turn me on Txt me your...
54      URGENT Your Mobile No was awarded a å£2000 Bon...
60      Back 2 work 2morro half term over Can U C me 2...
61      Sunshine Quiz Wkly Q Win a top Sony DVD player...
64      Valentines Day Special Win over å£1000 in our ...
68      FREE2DAY sexy St Georges Day pic of JordanTxt ...
71      Congrats Nokia 3650 video camera phone is your...
72      Hard LIVE 121 chat just 60pmin. Choose your gi...
80               You have 1 new message. Call 02070836089
82      complimentary 4 STAR Ibiza Holiday or å£10000 ...
85      YOU VE WON Your 4 Costa Del Sol Holiday or å£5...
89      Show ur colours Euro 2004 241 Offer Get an Eng...
94      Camera

In [133]:
lista_ham = np.sum(d_ham.Email+ " ").split()
vh= pd.Series(lista_ham)
lista_ham.count('to')

379

In [113]:
lista_spam = np.sum(d_spam.Email + " ").split()
vs= pd.Series(lista_spam)
vs.value_counts()
lista_spam.count('to')

184

In [130]:
lsp= len(lista_spam)
lha= len(lista_ham)
x = "Thanks for your subscription to Ringtone UK your mobile will be charged å£5/month Please confirm by replying YES or NO. If you reply NO you will not 0be charge"
def email(x):
    sep=x.split(' ')
    spams = []
    hams = []
    for i in range(len(sep)):
        spams.append((lista_spam.count(sep[i])/lsp))
        hams.append((lista_ham.count(sep[i])/lha))
    pps = np.sum(spams)
    pph = np.sum(hams)
    if pph > pps:
        return "Ham"
    else:
        return "Spam"
email(x)

'Spam'

___
## 4. Curva ROC

___
## 5. Qualidade do Classificador alterando a base de treinamento